# Tech Talk Tuesday: ETL's 
I have been developing the Scinamic Integration so I will use this one as the example. Fortunately, you can grab Scinamic data in batches of 1K. 

This took our ETL from 17 hours to several minutes.

In [1]:
from config import * # the user password and endpoint stored in config
from api import * # has the Scinamic_Compounds class etc.
import time # just for demonstrating speed of ETL

NameError: name 'image' is not defined

In [10]:
sci_session = Scinamic_Session(SCINAMIC_API_CONFIG)

# Clear Ways to Get Data from API
General SA Level Knowledge:
The classes below are all defined in api.py

In [11]:
compounds = Scinamic_Compounds(sci_session)
analysis = Scinamic_Analysis(sci_session)
projects = Scinamic_Projects(sci_session)
studies = Scinamic_Studies(sci_session)
results = Scinamic_Results(sci_session)
compoundbatches = Scinamic_CompoundBatches(sci_session)
curves = Scinamic_Curves(sci_session)

# Using "get_all_data" Function

Start with Compounds

In [14]:
start = time.time()
compounds.get_all_data()
print("Gathering All Compound Data Took: %.2f Seconds\nThere are %i Compounds"%(time.time()-start,len(compounds.pks)))

Gathering All Compound Data Took: 2.45 Seconds
There are 6509 Compounds


In [28]:
compounds.data[0]

{'pk': '7249403',
 'id': 'ALK-0005548',
 'creation_user': 'cquesnelle',
 'creation_date': '2023-05-04 16:48:02',
 'lastedit_user': '',
 'projects': 'A&CSP1',
 'mw': '843.9121',
 'exact_mass': '843.2911',
 'formula': 'C42H41N11O7S',
 'smiles': 'O=C(CCN1C=C(CCCCS(=O)(=O)C2=CC=CC(NC3=NC=C4C=CN=C(N5CCC(CC5)C#N)C4=C3)=C2)N=N1)NC1=CC=C2C(=O)N(C3CCC(=O)NC3=O)C(=O)C2=C1 |c:16,24,38,40,43,66,t:5,14,20,22,26,47,49|',
 'stereochemistry': 'RACEMIC',
 'lg_formula': 'NA',
 'hb_donor_count': '3',
 'ring_count': '8',
 'aromatic_ring_count': '6',
 'xlogp': '2.93',
 'alogp': '3.3698',
 'tpsa': '250.72',
 'lipinski_failures': '3',
 'active': 'true',
 'group': 'A&CSP1',
 'entity': 'Compound'}

# Results

In [31]:
start = time.time()
results.get_all_data()
print("Gathering All Result Data Took: %.2f Seconds\nThere are %i Results in Scinamic"%((time.time()-start),len(results.pks)))

Gathering All Compound Data Took: 30.05 Seconds
There are 76753 Results in Scinamic


In [32]:
results.data[0]

{'pk': '6966944',
 'compound': 'ALK-0003445',
 'compoundbatch': 'ALK-0003445-01',
 'study': 'IntactMS-SP',
 'value_type': 'adduct0_percent',
 'value_numeric': '100.0',
 'protein': 'NRAS.1',
 'status': 'valid',
 'project': 'NONE',
 'proteinbatch': 'NRAS.1-01',
 'timepoint': '24.0',
 'timepoint_unit': 'h',
 'user': 'seron',
 'analysis': 'A-2023-0040',
 'source': 'PureHoneyTech',
 'time': '2023-02-02 10:57:36',
 'compoundrequest': 'CR20221212-1',
 'entity': 'Result'}

# Curves W.I.P

Curves and other images should be handled the same*

There may be differences of processing images and curves, but we need a single standard to retrieve and store the images. I propose an "Image Service" opposed to curve serve. As LD continues with biologics we will be bringing in histology and other data types. A flask app that serves these images will be necessary. 

No matter the image holds the endpoint will be unique and it simply points at an image in a persistant volume (k8s)

This is a step that would require indexing for regular pulls.

In [48]:
import base64
from PIL import Image
import io
# comes back as string
start = time.time()
image_data = sci_session.get_curve(curves.pks[0]).data
decoded_data = base64.b64decode(image_data)
image_stream = io.BytesIO(decoded_data)
image = Image.open(image_stream)
# image.save('image.png')
image.show()
print("Generating Single Curve Took: %.2f Seconds\nThere are %i Curves in Scinamic, therefore the estimated time to render all curves: %.2f Minutes"%((time.time()-start),len(curves.pks),(len(curves.pks)*(time.time()-start))/60))

Generating Single Curve Took: 0.34 Seconds
There are 9335 Curves in Scinamic, therefore the estimated time to render all curves: 53.26 Minutes


# Map to SimpleSchema

In [4]:
simp_session = SimpleSchema_Session(SIMPLE_SCHEMA_DB_CONFIG)

In [33]:
# simp_session.create_tables()
simp_session.clear_all()

In [34]:
from map import * # We can change the name (suggestions?)
from scinamic_utils import *
add_projects_to_ss()

In [35]:
# compounds.get_all_data() # 3 seconds
start = time.time()
compound_map(compounds)
print("Mapping compounds took: %.2f Minutes"%((time.time()-start)/60))

[13:54:23] Explicit valence for atom # 1 N, 4, is greater than permitted


error creating molfile


[13:54:28] Explicit valence for atom # 1 N, 4, is greater than permitted


error creating molfile


[13:54:48] Explicit valence for atom # 1 N, 4, is greater than permitted
[13:54:48] Explicit valence for atom # 1 N, 4, is greater than permitted


error creating molfile
error creating molfile


[13:54:54] Explicit valence for atom # 1 N, 4, is greater than permitted


error creating molfile


[13:55:05] Explicit valence for atom # 1 N, 4, is greater than permitted


error creating molfile
Mapping compounds took: 2.48 Minutes


# Map Assay (Root from Results)

In [37]:
# results.get_all_data() # 30 seconds
start = time.time()
assay_map(results)
print("Mapping Assay Data took: %.2f Minutes"%((time.time()-start)/60))

Mapping Assay Data took: 8.34 Minutes
